# Introduction

In [ ]:
# The idea of this competition, is to predict sales for mnay product families for stores in ecuador.
#Train and test share: date, store_nbr, family(product_family), onpromotion (number of products of the family that were on promotion)
#Train also has the sales.
#Test is what we are trying to predict in the competition

#Stores contains store metadata, we might want to join that data into our train and test tables to improve predictions if they are not redundant with the store_nbr (aka if at least some of the stores share their values).
 #stores metadata
#type and cluster are not redundant. see stores_df[['type', 'cluster']].drop_duplicates()#.pivot(columns='cluster')

#Daily oil price, keep in mind oil price affects ecuador economical health.
#We probably want to join it into our dataframe, but keep in mind it seems to have some null values. We probably want to fill downwards so that it takes its value from the previous date but well have to see. We could also average between the next and previous day.


#While its not clear it seems to be the total transactions of the store on that day


#A table containing holiday events. 
#its worth noting that some events were transfered in date so we need to take into account hte actual date. transfered column True means that the holiday wasnt actually celebrated that date.
#Also, keep in mind that some seem to be regional and some local. This could mean that we need to take into account the region in considering them.
#Some special_days were of type bridge, meaning  they are extra days added to the same holiday.
#This are often compensated by making some not working days(ie saturday) working days. This are of type workday.
#holiday_type additional means that it isnt an actual holiday, but actually the extension of one.

#Additonally to these tables wages on the public sector are paid on the 15th and last day of each month which could affect supermarket sales
#A great earthquiake struck ecuador on april 16 2016 which resulted in donations possibly affecting sales.


#TLDR we need to add many features to the train and test df.
#First lets make some column names easier to understand. Specially after joining.

In [ ]:
import os
import json
with open('./.kaggle/kaggle.json') as credentials_file:
    credentials_dict = json.load(credentials_file)
    os.environ['KAGGLE_USERNAME'] = credentials_dict['username']
    os.environ['KAGGLE_KEY'] = credentials_dict['key']
    import kaggle
from zipfile import ZipFile
import pandas as pd
import tensorflow as tf
import keras
from tqdm import tqdm
import re
from pandas.tseries.offsets import MonthEnd
from scipy.stats import shapiro
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
pd.set_option('display.max_rows', 1000)
#TODO move the eda process to another file? Might need to move the processing functions to an auxiliary file aswell.

In [ ]:
def download_dataset():
    kaggle.api.competition_download_files(competition='store-sales-time-series-forecasting', path='./dataset', force=False, quiet=True)
    with ZipFile('./dataset/store-sales-time-series-forecasting.zip') as dataset_zip:
        dataset_zip.extractall('./dataset')

def get_raw_dfs() ->tuple:
    """Get dataframes containing all the info in the dataset."""
    train_df_inc = pd.read_csv('./dataset/train.csv', index_col='id')
    test_df_inc = pd.read_csv('./dataset/test.csv', index_col='id')
    stores_df = pd.read_csv('./dataset/stores.csv', index_col='store_nbr')
    oil_df = pd.read_csv('./dataset/oil.csv') 
    transactions_df = pd.read_csv('./dataset/transactions.csv')
    holiday_events_df = pd.read_csv('./dataset/holidays_events.csv')
    sample_submission_df = pd.read_csv('./dataset/sample_submission.csv')
    return train_df_inc, test_df_inc, stores_df, oil_df, transactions_df, holiday_events_df, sample_submission_df

def rename_raw_dfs_cols(train_df_inc, test_df_inc, stores_df, oil_df, transactions_df, holiday_events_df, sample_submission_df) ->tuple:
    """Rename the columns of the raw dataframes so they are more easily understandable"""
    train_df_inc = train_df_inc.rename(columns={'family':'product_family', 'onpromotion':'products_of_family_on_promotion'})
    test_df_inc = test_df_inc.rename(columns={'family':'product_family'})
    oil_df = oil_df.rename(columns={'dcoilwtico':'oil_price'})
    stores_df = stores_df.rename(columns={'type':'store_type', 'cluster':'store_cluster', 'city':'store_city', 'state':'store_state'})
    transactions_df = transactions_df.rename(columns={'transactions':'all_products_transactions'})
    special_days_df = holiday_events_df.rename(columns={'type':'day_type', 'locale':'special_day_locale_type', 'locale_name':'special_day_locale','description':'special_day_reason', 'transferred':'special_day_transferred'})
    return train_df_inc, test_df_inc, stores_df, oil_df, transactions_df, special_days_df, sample_submission_df

def complete_features_dataset(features_df:pd.DataFrame, stores_df:pd.DataFrame, oil_df:pd.DataFrame, transactions_df:pd.DataFrame, special_days_df:pd.DataFrame)->pd.DataFrame:
    """Add relevant columns from the auxiliary dataframes into a features dataset, be it the train set or the test set."""
    full_features_df = features_df.merge(oil_df, on='date',how='left')
    full_features_df = full_features_df.merge(stores_df, on=['store_nbr'], how='left') #Need to also consider date. 'date', nvm
    full_features_df = full_features_df.merge(special_days_df, on='date', how='left')
    full_features_df = full_features_df.merge(transactions_df, on=['date', 'store_nbr'], how='left')
    return full_features_df

def reorder_features_dataset(features_df):
    """Reorder the columns in the feature dataframe so the table becomes easier to understand and inspect. Does not affect the rows."""
    return features_df # TODO Order to columns more logically

def one_hot_encode_necessary_features(features_df): #TODO change call to after the creation of the proper columns, and the columns to match it.
    """One hot encodes the columns, using their name as prefix, and adding them into the same place the original was, while removing the original """
    names_of_columns_to_ohe= [ 'store_nbr', 'product_family', 'store_city','store_state',
                      'store_type', 'day_type', 'special_day_locale_type', 'special_day_locale',
                       'special_day_reason', 'special_day_transferred']
                    #holiday_transferred may be better vectorized. Day type might be too. holiday_locale_type too.
                    #This is because their values might have meaning in the order.
    
    for name_of_column_to_ohe in names_of_columns_to_ohe:
        index_of_col_to_ohe = features_df.columns.get_loc(name_of_column_to_ohe)
        one_hot_encoded_column:pd.DataFrame = pd.get_dummies(features_df[name_of_column_to_ohe], dummy_na=True, prefix=name_of_column_to_ohe)
        
        # Split the dataframe into two parts: before and after the position of the original column
        columns_before_col_to_ohe = features_df.iloc[:, :index_of_col_to_ohe]
        columns_after_col_to_ohe = features_df.iloc[:, index_of_col_to_ohe+1:] #Exclude the current column

        # Concatenate the two parts with the one-hot encoded dataframe in between
        features_df = pd.concat([columns_before_col_to_ohe, one_hot_encoded_column, columns_after_col_to_ohe], axis=1)

    return features_df
    #TODO #return the dataset and the references

def process_numerical_features(features_df, normalizers, is_test_data): #Should take the normalizer as parameter.
    """Either normalize or standardize variables depending on their distribution. Also handle missing values where necessary. """
    print(features_df.columns)
    numerical_variable_names = ['oil_price', 'all_products_transactions']

    #Normalization is a scaling technique in which values are shifted and rescaled so that they end up ranging between 0 and 1. It is also known as Min-Max scaling.
    #Its more robust than standarization since it doesnt require the distribution to be gaussian.
    #MinMaxScaler is a scikitlearn normalizer.

    for variable_name in numerical_variable_names: #TODO MAKE IT CHECK IF ITS TEST AND TRANSFORM INSTEAD OF FIT TRANSFORM
        features_df[variable_name + '_standardized'] = normalizers[variable_name].fit_transform(features_df[variable_name])
        features_df[variable_name + '_normalized'] = normalizers[variable_name].fit_transform(features_df[variable_name]) #hadd double brakcets


    return features_df
    #TODO #return the dataset and the normalizer

def fill_missing_values(features_df):
    """ """

def fill_missing_oil_values(features_df):
    # Oil price highly varies over time, making average imputing inaddecuate.
    # While we could use the average between the previous day and the next day, that wouldnt work for forecasting.
    # Instead just use the last known price, sinceFill down
    features_df['oil_price'] = features_df['oil_price'].fillna(method='ffill')
    return features_df


def refine_special_day_reason(features_df):
    """Assign the same reason to special days that have the same reason but with a small variation, storing otherwise lost information in other columns"""

    # Create a special_day_reason subtype column. Extract the subtype of mundial de futbol brasil into a subtype column, since they all have a common reason.
    features_df['special_day_reason_subtype'] = features_df['special_day_reason'].apply(lambda x: x.replace('Mundial de futbol Brasil: ', '').replace('Mundial de futbol Brasil', '') if (type(x) == str) and (x.startswith('Mundial de futbol Brasil')) else '')
    
    # Create a function to extract the offset in any reasons that end with -number or +number
    # Function to process the special_day_offset column
    def get_special_day_offset(value):
        if type(value) == str:
            match = re.search(r'([-+]\d+)$', value)
            if match:
                # Return the number with the sign as an integer
                return int(match.group())

            # If the condition is not met, return NaN
        return 0
    
    #Use the function to create a special_day_offset_column
    features_df['special_day_offset'] = features_df['special_day_reason'].apply(get_special_day_offset)


    # Now that the data that is not stored elsewhere has been stored create a function to clean the special_day_reason column
    def process_special_day_reason_value(value):
        '''A function to process the special_day_reason elements that can be applied element-wise using pandas.'''

        if type(value) == str:
            #If the reason contains a locale, eliminate the locale part of the reason, since its already stored in locale column.
            prefixes = ['Fundacion', 'Independencia', 'Provincializacion', 'Cantonizacion', 'Translado']
            for prefix in prefixes:
                if value.startswith(prefix):
                    value = prefix
            
            # If if value ends with - followed by any number or + followed by any number, aka if it has an offset, eliminate the offset part of the reason, since its already stored in the offset column.  
            match = re.search(r'([-+]\d+)$', value) # Check if value ends with - followed by any number or + followed by any number
            if match:
                return value.replace(match.group(), '') # Return the string without the number and sign
            
            # If the reason is mundial de futbol brasil, eliminate additional detaisl since they are already stored in the reason_subtype column. #Todo verify synthax
            if 'Mundial de futbol Brasil' in value: #value.contains('Mundial de futbol Brasil'):
                return 'Mundial de futbol Brasil'
        
        # If the reason does not need any preprocessing just return it
        return value

    #And now apply that function element-wise to cleanup the special_day_reason column to not store redundant info and unify reasons that are actually the same.
    features_df['special_day_reason'] = features_df['special_day_reason'].apply(process_special_day_reason_value)

    return features_df


def replace_date_with_date_related_columns(features_df):
    """"Adds many date_related_columns """

    features_df['date'] = pd.to_datetime(features_df['date'])

    #Calculate the absolute date (from unix epoch), which is a standard start date used by many siystems and libraries working with time series data.
    #While this would work its probably worth for the nn to learn.
    #UNIX_EPOCH = pd.Timestamp("1970-01-01")
    #features_df['absolute_day_number'] = (features_df['date'] - UNIX_EPOCH) // pd.Timedelta('1D')

    #Calculate the number of days since the first date in the dataset.
    features_df['df_start_absolute_day'] = (features_df['date'] - features_df['date'].min()) // pd.Timedelta('1D')

    features_df['day_of_year'] = features_df['date'].dt.dayofyear
    features_df['day_of_month'] = features_df['date'].dt.day
    features_df['day_of_week'] = features_df['date'].dt.dayofweek

    features_df['is_15th'] = (features_df['date'].dt.day == 15).astype(int)
    features_df['is_last_day_of_month'] = (features_df['date'] == features_df['date'] + MonthEnd(1)).astype(int)

    features_df = features_df.drop(columns=['date'])



def prepare_features_df(features_df:pd.DataFrame, stores_df, oil_df, transactions_df, special_days_df, normalizers):
    """Call all the preprocessing methods in order, and prepare a features dataframe for deep learning, be it the train set or the test set"""
    #TODO READ and rename columns of the dataset, and then process them here.
    features_df = complete_features_dataset(features_df, stores_df, oil_df, transactions_df, special_days_df) #Careful, do not move this later, since the rest of the methods need access to all the columns.
    features_df = reorder_features_dataset(features_df) #Careful, do not move this later, the rest of the methods try to respect the order of the dataframe, so theyll end up in messy places otherwise.
    features_df = fill_missing_oil_values(features_df)     
    features_df = refine_special_day_reason(features_df)
    features_df = process_numerical_features(features_df) #Rename or include fill missing oil values here?
    features_df = one_hot_encode_necessary_features(features_df) #Careful moving this earlier since one hot encoding properly requires categorical variables to have been processed.
    features_df = replace_date_with_date_related_columns(features_df) #Careful, moving calling this earlier could be problematic since it eliminates date column.

    return features_df

def window_dataset():
    """Window the dataset so that it can be used for training a neural network."""
    #Aka create a window of days with values and then the unknown value for the next day. and then slide that window forward to create another data point.
    #TODO
    return

def remove_seasonality():
    #perform fourier transform? 
    #TODO?
    return

def train_val_split_dataset():
    """Split the datataset so that it can be used for training and then validation"""
    #TODO, probably the most sensible approach for a simple time series analysis is to use the last data for validation,
    #though i can imagine other approaches which could lead to better peformance, for example masking like bert does for text data.
    #Or creating crossval sets by skipping part of the time before prediction
    return

def prepare_datasets():
    train_df_inc, test_df_inc, stores_df, oil_df, transactions_df, special_days_df, sample_submission_df = rename_raw_dfs_cols(*get_raw_dfs())
    train_y_truth = train_df_inc.pop('sales')
    normalizers = {'oil_price':MinMaxScaler(), 'all_products_transactions':MinMaxScaler()}
    train_df, test_df = prepare_features_df(train_df_inc, stores_df, oil_df, transactions_df, special_days_df, normalizers, False), prepare_features_df(test_df_inc, stores_df, oil_df, transactions_df, special_days_df, normalizers, True)
    return train_df, test_df, train_y_truth, sample_submission_df


train_df, test_df, train_y_truth, sample_dataset = prepare_datasets()